<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# OpenPIV on the bridgepile_wake

See the post on LinkedIn by Stefano Brizzolara
https://www.linkedin.com/posts/stefano-brizzolara-6a8501198_rheinfall-flowvisualization-ugcPost-6672832128742408192-lRub

In [1]:
from openpiv import tools, pyprocess, validation, filters, scaling 

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import imageio

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
frame_a  = tools.imread('../test8/frame0001.tif')
frame_b  = tools.imread('../test8/frame0002.tif')

FileNotFoundError: No such file: '/home/user/Documents/repos/test8/frame0001.tif'

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(12,10))
ax[0].imshow(frame_a,cmap=plt.cm.gray)
ax[1].imshow(frame_b,cmap=plt.cm.gray)

In [ ]:
# %pdb
# np.seterr(all="raise")
winsize = 24 # pixels
searchsize = 48  # pixels, search in image B
overlap = 12 # pixels
dt = 1./30 # sec, assume 30 fps

frame_a[:600,:] = 0  # basically masking out the non-illuminated region
frame_b[:600,:] = 0


u0, v0, sig2noise = pyprocess.extended_search_area_piv(frame_a.astype(np.int32), 
                                                       frame_b.astype(np.int32), 
                                                       window_size=winsize, 
                                                       overlap=overlap, dt=dt, 
                                                       search_area_size=searchsize, 
                                                       sig2noise_method='peak2peak',
                                                      correlation_method='linear',
                                                      normalized_correlation=True)

In [ ]:
x, y = pyprocess.get_coordinates(frame_a.shape,searchsize, overlap)

In [ ]:
u1, v1, mask = validation.sig2noise_val( u0, v0, sig2noise, threshold = 1.2)

In [ ]:
u2, v2 = filters.replace_outliers( u1, v1, method='localmean', max_iter=1, kernel_size=3)

In [ ]:
# x, y, u3, v3 = scaling.uniform(x, y, u2, v2, scaling_factor = 1. )

In [ ]:
x, y, u2, v2 = tools.transform_coordinates(x, y, u2, v2)
tools.save(x, y, u2, v2, mask, 'exp1_001.txt' )

In [ ]:
# tools.display_vector_field('exp1_001.txt', scaling_factor=100., width=0.0025)

In [ ]:
# If you need a larger view:
fig, ax = plt.subplots(figsize=(12,12))
tools.display_vector_field('exp1_001.txt', ax=ax, scaling_factor=1.0, 
                           scale=1000, width=0.0045, on_img=True, 
                           image_name='../test8/frame0001.tif');

In [ ]:
from openpiv.windef import Settings, Multiprocesser, piv

In [ ]:
settings = Settings()
settings.filepath_images = '../test8'
settings.frame_pattern_a = 'frame00*.tif'
settings.frame_pattern_b = '(1+2),(2+3)'
piv(settings)
